In [182]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

from bs4 import BeautifulSoup as bs
from datetime import datetime as dt
import time

import re

import pandas as pd

from pprint import pprint

In [25]:
def init_driver():
    driver = webdriver.Chrome()
    driver.wait = WebDriverWait(driver, 5)
    return driver

In [10]:
def close_driver(driver):
    driver.close()

In [42]:
def login_twitter(driver, username, password):
    
    driver.get('https://twitter.com/login')
    
    #identify username and password fields
    username_field = driver.find_element_by_class_name('js-username-field')
    password_field = driver.find_element_by_class_name('js-password-field')
    
    #'type in' username and password and wait 1 second
    username_field.send_keys(username)
    driver.implicitly_wait(1)
    password_field.send_keys(password)
    driver.implicitly_wait(1)
    
    #click login button
    driver.find_element_by_class_name('EdgeButtom--medium').click()
    
    return
    

In [70]:
class wait_for_more_than_n_elements_to_be_present(object):
    def __init__(self, locator, count):
        self.locator = locator
        self.count = count
    
    def __call__(self, driver):
        try:
            elements = EC._find_elements(driver, self.locator)
            return len(elements) > self.count
        except StaleElementReferenceException:
            return False
        

In [78]:
def search_twitter(driver, query, max_tweets):
    #find query box
    box = driver.wait.until(EC.presence_of_element_located((By.NAME, 'q')))
    #put cursor in search box and clear text
    driver.find_element_by_name('q').clear()
    
    #put query into box
    box.send_keys(query)
    box.submit()
    
    wait = WebDriverWait(driver, 10)
    
    try:
        time.sleep(2)
        wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'li[data-item-id]')))
        
        number_of_tweets = 0
        while number_of_tweets < max_tweets:
            
            #makes list of all tweets on page
            tweets = driver.find_elements_by_css_selector('li[data-item-id]')
            
            number_of_tweets = len(tweets)
            #this scrolls to the last tweet in the list, loading more tweets
            driver.execute_script('arguments[0].scrollIntoView();', tweets[-1])
            
            try:
                wait.until(wait_for_more_than_n_elements_to_be_present((By.CSS_SELECTOR, 'li[data-item-id]'),number_of_tweets))
            
            except TimeoutException:
                break
        
        page_source = driver.page_source
        
        
    except TimeoutException:
        page_source = None
    
    return page_source
        

In [246]:
driver = init_driver()
username = ''
password = ''
query = '#ENGvSL'

login_twitter(driver, username, password)

page_source = search_twitter(driver, query, 100)
#time.sleep(10)

#close_driver(driver)

In [247]:
soup = bs(page_source, 'html.parser')
#section of page containing tweet stream
tweet_stream = soup.find('div', class_='stream')

In [248]:
namelist = []

for li in tweet_stream.findAll('li', class_='js-stream-item'):
    if 'data-item-id' not in li.attrs:
        continue
    else:
        tweet = {}
        
        if li.find('div', class_='tweet') is not None:
            tweet['author'] = li.find('div', class_='tweet')['data-name']
        
        if li.find('div', class_='tweet') is not None:
            tweet['handle'] = li.find('div', class_='tweet')['data-screen-name']
        
        if li.find('p', class_='tweet-text') is not None:
            tweet['text'] = li.find('p', class_='tweet-text').text.strip()
        
        tweet['id'] = li['data-item-id']
        
        if li.find('span', class_='_timestamp') is not None:
            tweet_timestamp = int(t.find('span', class_='_timestamp')['data-time'])
            tweet['time'] = dt.fromtimestamp(tweet_timestamp).strftime("%Y/%m/%d %I:%M:%S")
    
    namelist.append(tweet)

tweets_df = pd.DataFrame(namelist)

tweets_df

,author,handle,id,text,time
0,Rahul Prakash,RahulPrakash9,1059785526251798528,Disappointed with @ECB_cricket : why drop @chr...,2018/11/07 01:49:46
1,Joe Wilson,joe_wils,1059891411397566465,I bet @benfoakes93 has done some shaggin in hi...,2018/11/07 01:49:46
2,SirDonBradman,BradmanSir,1060190773520547846,"Adil Rashid is a great bowler, Root should giv...",2018/11/07 01:49:46
3,AI,automatedintel,1059766031726579712,Our customers tell us it can take them over 40...,2018/11/07 01:49:46
4,James,jiffer9,1060076404493221888,When will this @Cricket_TS agony end. Why depr...,2018/11/07 01:49:46
5,Saadkhan,Saadkha66286650,1060180121557057537,#ENGvSL best couple of days after a long time...,2018/11/07 01:49:46
6,Richard May,rich_may,1060035568720056322,At least being up at stupid o clock for this c...,2018/11/07 01:49:46
7,Mike Kursar,MikeKursar,1060187020474150912,"Currently adding ""going on tour with @TheBarmy...",2018/11/07 01:49:46
8,Adam Dennant,D3NN4NT,1059718151192096768,Early morning gym session @FairlawnsHotel watc...,2018/11/07 01:49:46
9,HowzStat,HowzStat,1060084554487095297,Ben Foakes has become the 105th batsman - and ...,2018/11/07 01:49:46
